In [78]:
import json
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
#Set two decimal points for display
pd.options.display.float_format = "{:.2f}".format
                        

In [2]:
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot"

In [3]:
all3= pd.read_csv('C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/LWL & EF Tobii Data Export/Baby Toolbox_Data_Export_06_02_2021.tsv', sep='\t')

In [4]:
ada = pd.read_csv('C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/LWL & EF Tobii Data Export/Ada LWL Only/Baby Toolbox_Data_Export_Ada_LWLonly.tsv', sep='\t')

In [5]:
data_dir2='C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/LWL & EF Tobii Data Export/Second Attempts LWL Only'

In [6]:
os.chdir(data_dir2)

In [7]:
ada2= pd.read_csv('Baby Toolbox_Data_Export_Ada LWL Take 2.tsv', sep='\t') 

In [8]:
claire2= pd.read_csv('Baby Toolbox_Data_Export_Claire LWL Take 2.tsv', sep='\t') 

In [9]:
claire2.head()

,ExportDate,StudioVersionRec,StudioProjectName,StudioTestName,ParticipantName,RecordingName,RecordingDate,RecordingDuration,RecordingResolution,PresentationSequence,...,DistanceLeft,DistanceRight,PupilLeft,PupilRight,ValidityLeft,ValidityRight,IRMarkerCount,IRMarkerID,PupilGlassesRight,Unnamed: 173
0,6/30/2021,3.4.8,Baby Toolbox,LWL website,ClaireJune2,Rec 10,6/23/2021,303996,1920 x 1080,NaN,...,462.43,462.43,-1.00,-1.00,0.0,0.0,NaN,NaN,NaN,NaN
1,6/30/2021,3.4.8,Baby Toolbox,LWL website,ClaireJune2,Rec 10,6/23/2021,303996,1920 x 1080,NaN,...,462.35,462.35,-1.00,-1.00,0.0,0.0,NaN,NaN,NaN,NaN
2,6/30/2021,3.4.8,Baby Toolbox,LWL website,ClaireJune2,Rec 10,6/23/2021,303996,1920 x 1080,NaN,...,462.33,462.33,3.36,3.06,0.0,0.0,NaN,NaN,NaN,NaN
3,6/30/2021,3.4.8,Baby Toolbox,LWL website,ClaireJune2,Rec 10,6/23/2021,303996,1920 x 1080,NaN,...,462.14,462.14,-1.00,-1.00,0.0,0.0,NaN,NaN,NaN,NaN
4,6/30/2021,3.4.8,Baby Toolbox,LWL website,ClaireJune2,Rec 10,6/23/2021,303996,1920 x 1080,NaN,...,462.07,462.07,-1.00,-1.00,0.0,0.0,NaN,NaN,NaN,NaN


In [10]:
doc=[]
doc.append(ada)
doc.append(ada2)
doc.append(claire2)
doc.append(all3)
tdf = pd.concat(doc)

In [11]:
tdf.shape

(411928, 381)

In [12]:
keep = [col for col in tdf.columns if 'AOI' in col]
keep = [col for col in keep if 'Calib' not in col]

In [13]:
k24 = keep[4:76]

In [14]:
names={'P03':'Ada2', 'ClaireJune2':'ClaireJune2', 'ClaireP1':'ClaireP1', 'BrianP2':'BrianP2', 'SKJ_P2':'SKJ_P2','Adatake2':'Ada'}
tdf['ParticipantName']=tdf['ParticipantName'].map(names)

In [15]:
tdf['ParticipantName'].value_counts()

ClaireP1       107784
BrianP2        101390
SKJ_P2          96213
Ada2            36389
ClaireJune2     36145
Ada             34007
Name: ParticipantName, dtype: int64

# Read json file

In [16]:
import json
import os
import pandas as pd

In [17]:
#claire
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Tobii Verbose Files/Claire"
os.chdir(data_dir)
df=pd.read_json('949BCA6A-D3D6-426C-9A94-E806F84E4E72@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00024@2021-05-11-22.11.48.356@LookListening@Claire@VerboseGazeData.json')

In [18]:
data_dir="C:/Users/tsb7592/Downloads"
os.chdir(data_dir)

In [19]:
#second attempt only
cdf=pd.read_json('2021-06-23-22.58.27.217@LookListening@Claire@VerboseGazeData.json')

In [20]:
cols=['itemID','eventName','gazeLocationName', 'gazeLocationOnScreen', 'arHitTestLookAtPointPositionX', 'arHitTestLookAtPointPositionY',"lookAtPointX", "lookAtPointY",]

# Clean the data for machine learning

In [21]:
cols=['itemID','eventName', 'gazeLocationName','gazeLocationOnScreen',
      'arHitTestLookAtPointPositionX', 'arHitTestLookAtPointPositionY',
 'lookAtPointX', 'lookAtPointY','NewLocation','NewOnScreen']

In [142]:
def combine_tsv_json(tdf, df, person, k24):
    useX = "lookAtPointX"
    useY = "lookAtPointY"
    useX1 = "arHitTestLookAtPointPositionX"
    useY1 = "arHitTestLookAtPointPositionY"
        
    ct = tdf[tdf['ParticipantName']==person][k24]
    begin =list(df[df['eventName'].str.contains('presentedLiveItem')].index)
    end = list(df[df['eventName'].str.contains('completedLiveItem')].index)

    rv = []
    for i in range(24):
        lcol,rcol,wcol = k24[3*i:3*i+3]
        sub = ct[ct[wcol].notnull()]
    
        idx =[]
        mni =df.iloc[begin[i]:end[i]][[useX, useY,useX1, useY1]]
        mni =mni[mni[useX].notnull()]

        My_list = [*range(mni.shape[0])]
        ratio = round(sub.shape[0]/mni.shape[0], 2) - 0.03
        idx = [int(round(element * ratio)) for element in My_list]
        sub = sub.iloc[idx].reset_index()
        sub = sub[[lcol,rcol,wcol]]
        js =  mni.reset_index()
        sub.columns = ['Left','Right','Wholescreen']
        result = pd.concat([sub, js[[useX, useY, useX1, useY1]] ], axis =1)
        result['item'] = i
        rv.append(result)
    
    data = pd.concat(rv)
    
    #Create a Y variable base on Left, right, onscreen, offscreen
    data.loc[(data['Left'] == 1), 'Y'] = 'Left'
    data.loc[(data['Right'] == 1), 'Y'] = 'Right'
    data.loc[((data['Left'] == 0) & (data['Right'] ==0)& (data['Wholescreen'] ==0)), 'Y'] = 'OffAway'
    data.loc[((data['Left'] == 0) & (data['Right'] ==0)& (data['Wholescreen'] ==1)), 'Y'] = 'OnAway'
    
    return data[['Y', useX, useY, useX1, useY1]]

In [143]:
c1  = combine_tsv_json(tdf, df, 'ClaireP1', k24)
c2  = combine_tsv_json(tdf, cdf, 'ClaireJune2', k24)

In [132]:
c1a  = combine_tsv_json(tdf, df, 'ClaireP1', k24, 'arHitTestLookAtPointPosition')
c2a  = combine_tsv_json(tdf, cdf, 'ClaireJune2', k24, 'arHitTestLookAtPointPosition')

In [85]:
c1.Y.value_counts()

Left       5776
Right      4186
OffAway    1103
OnAway      413
Name: Y, dtype: int64

In [164]:
# example of multi-class classification task
from numpy import where
from collections import Counter
from sklearn.datasets import make_blobs
from matplotlib import pyplot

from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier




def own_test(df):
    X = df.iloc[:,1:]
    y = df['Y']
    x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = True, test_size = 0.25) 
    #Create a Gaussian Classifier
    print('RandomForest')
    clf=RandomForestClassifier(n_estimators=100)

    #Train the model using the training sets y_pred
    y_pred =clf.fit(x_train, y_train).predict(x_test)
    rv = pd.DataFrame(metrics.classification_report(y_test, y_pred, digits=3,output_dict=True)).T

    rv['support'] = rv.support.apply(int)
    display(rv.style.background_gradient(cmap='coolwarm',
                             subset=pd.IndexSlice['Left':'accuracy','f1-score']))
    #svc
    print('SupportVector')
    clf = SVC(probability=False, kernel="rbf", C=2.8, gamma=0.0073)  # cache_size=200,
    y_pred = clf.fit(x_train, y_train).predict(x_test)

    rv = pd.DataFrame(metrics.classification_report(y_test, y_pred, digits=3,output_dict=True)).T

    rv['support'] = rv.support.apply(int)
    display(rv.style.background_gradient(cmap='coolwarm',
                             subset=pd.IndexSlice['Left':'accuracy','f1-score']))
    
    #Create a Gaussian Classifier
    print('NaiveBayes')
    gnb = GaussianNB()
    #Predict the response for test dataset
    y_pred = gnb.fit(x_train, y_train).predict(x_test)
    
    rv = pd.DataFrame(metrics.classification_report(y_test, y_pred, digits=3,output_dict=True)).T

    rv['support'] = rv.support.apply(int)
    display(rv.style.background_gradient(cmap='coolwarm',
                             subset=pd.IndexSlice['Left':'accuracy','f1-score']))
    
    print('knn')
    #Create KNN Classifier
    knn = KNeighborsClassifier(n_neighbors=5)

    #Train the model using the training sets
    y_pred = knn.fit(x_train, y_train).predict(x_test)
    rv = pd.DataFrame(metrics.classification_report(y_test, y_pred, digits=3,output_dict=True)).T

    rv['support'] = rv.support.apply(int)
    display(rv.style.background_gradient(cmap='coolwarm',
                             subset=pd.IndexSlice['Left':'accuracy','f1-score']))
    

In [162]:
def twoset_test(df1, df2):
    x_train= df1.iloc[:,1:]
    y_train = df1['Y']
    x_test= df2.iloc[:,1:]
    y_test = df2['Y']

    clf = SVC(probability=False, kernel="rbf", C=2.8, gamma=0.0073)  # cache_size=200,
    y_pred = clf.fit(x_train, y_train).predict(x_test)

    rv = pd.DataFrame(metrics.classification_report(y_test, y_pred, digits=3,output_dict=True)).T
    
    rv['support'] = rv.support.apply(int)
    display(rv.style.background_gradient(cmap='coolwarm',
                             subset=pd.IndexSlice['Left':'accuracy','f1-score']))

In [165]:
#Claire first attempt
rv =own_test(c1) 

RandomForest


,precision,recall,f1-score,support
Left,0.791423,0.849965,0.819650,1433
OffAway,0.664948,0.472527,0.552463,273
OnAway,0.634146,0.433333,0.514851,120
Right,0.781043,0.789272,0.785136,1044
accuracy,0.774564,0.774564,0.774564,0
macro avg,0.717890,0.636274,0.668025,2870
weighted avg,0.769040,0.774564,0.768935,2870


SupportVector


,precision,recall,f1-score,support
Left,0.740625,0.826936,0.781405,1433
OffAway,0.595238,0.274725,0.375940,273
OnAway,0.619048,0.216667,0.320988,120
Right,0.721416,0.761494,0.740913,1044
accuracy,0.725087,0.725087,0.725087,0
macro avg,0.669082,0.519956,0.554811,2870
weighted avg,0.714724,0.725087,0.708856,2870


NaiveBayes


,precision,recall,f1-score,support
Left,0.717618,0.815771,0.763553,1433
OffAway,0.336735,0.120879,0.177898,273
OnAway,0.333333,0.008333,0.016260,120
Right,0.689474,0.752874,0.719780,1044
accuracy,0.693031,0.693031,0.693031,0
macro avg,0.519290,0.424464,0.419373,2870
weighted avg,0.655082,0.693031,0.660676,2870


knn


,precision,recall,f1-score,support
Left,0.775288,0.845080,0.808681,1433
OffAway,0.554113,0.468864,0.507937,273
OnAway,0.602564,0.391667,0.474747,120
Right,0.784785,0.750958,0.767499,1044
accuracy,0.756098,0.756098,0.756098,0
macro avg,0.679187,0.614142,0.639716,2870
weighted avg,0.750482,0.756098,0.751131,2870


In [153]:
#Cross Validation for Claire using first attempt as train, second attempt as test
twoset_test(c1,c2)

,precision,recall,f1-score,support
Left,0.655145,0.832698,0.733327,4985
OffAway,0.157233,0.055107,0.081610,1361
OnAway,0.159574,0.080645,0.107143,186
Right,0.774047,0.715889,0.743833,4991
accuracy,0.678122,0.678122,0.678122,0
macro avg,0.436500,0.421085,0.416478,11523
weighted avg,0.639837,0.678122,0.650795,11523


In [99]:
#Sheila
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Tobii Verbose Files/Sheila"
os.chdir(data_dir)
sdf=pd.read_json('B8C654C6-5EAA-452E-8A80-B0281F7533C8@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00030@2021-05-12-00.31.22.201@LookListening@SKJ@VerboseGazeData.json')

In [147]:
s1  = combine_tsv_json(tdf, sdf, 'SKJ_P2', k24)

In [166]:
#cross validation for Sheila
own_test(s1)

RandomForest


,precision,recall,f1-score,support
Left,0.840288,0.877536,0.858508,1331
OffAway,0.621795,0.404167,0.489899,240
OnAway,0.714286,0.504587,0.591398,109
Right,0.824841,0.864053,0.843992,1199
accuracy,0.818340,0.818340,0.818340,0
macro avg,0.750302,0.662586,0.695949,2879
weighted avg,0.810870,0.818340,0.811622,2879


SupportVector


,precision,recall,f1-score,support
Left,0.777400,0.858002,0.815714,1331
OffAway,0.666667,0.041667,0.078431,240
OnAway,0.526316,0.091743,0.156250,109
Right,0.746366,0.856547,0.797670,1199
accuracy,0.760333,0.760333,0.760333,0
macro avg,0.679187,0.461990,0.462016,2879
weighted avg,0.745738,0.760333,0.721770,2879


NaiveBayes


,precision,recall,f1-score,support
Left,0.764510,0.861007,0.809894,1331
OffAway,0.571429,0.016667,0.032389,240
OnAway,0.000000,0.000000,0.000000,109
Right,0.742170,0.849875,0.792379,1199
accuracy,0.753387,0.753387,0.753387,0
macro avg,0.519527,0.431887,0.408666,2879
weighted avg,0.710166,0.753387,0.707122,2879


knn


,precision,recall,f1-score,support
Left,0.820225,0.877536,0.847913,1331
OffAway,0.606897,0.366667,0.457143,240
OnAway,0.631579,0.440367,0.518919,109
Right,0.819287,0.843203,0.831073,1199
accuracy,0.804099,0.804099,0.804099,0
macro avg,0.719497,0.631943,0.663762,2879
weighted avg,0.794908,0.804099,0.795868,2879


In [149]:
#Ada
data_dir="C:/Users/tsb7592/Downloads"
os.chdir(data_dir)
adf=pd.read_json('Ada@VerboseGazeData.json')
adf2=pd.read_json('2021-06-24-12.42.35.513@LookListening@Ada@VerboseGazeData 2.json')
a1  = combine_tsv_json(tdf, adf, 'Ada', k24)
a2  = combine_tsv_json(tdf, adf2, 'Ada2', k24)

In [167]:
#cross validation for Ada
own_test(a1)

RandomForest


,precision,recall,f1-score,support
Left,0.728814,0.735043,0.731915,351
OffAway,0.908209,0.918901,0.913524,1529
OnAway,0.566265,0.447619,0.500000,105
Right,0.759191,0.760589,0.759890,543
accuracy,0.839794,0.839794,0.839794,0
macro avg,0.740620,0.715538,0.726332,2528
weighted avg,0.837090,0.839794,0.838133,2528


SupportVector


,precision,recall,f1-score,support
Left,0.574344,0.561254,0.567723,351
OffAway,0.845629,0.892086,0.868237,1529
OnAway,0.375000,0.171429,0.235294,105
Right,0.667939,0.644567,0.656045,543
accuracy,0.763054,0.763054,0.763054,0
macro avg,0.615728,0.567334,0.581825,2528
weighted avg,0.750248,0.763054,0.754645,2528


NaiveBayes


,precision,recall,f1-score,support
Left,0.150538,0.997151,0.261584,351
OffAway,0.983871,0.119686,0.213411,1529
OnAway,0.235294,0.038095,0.065574,105
Right,0.000000,0.000000,0.000000,543
accuracy,0.212421,0.212421,0.212421,0
macro avg,0.342426,0.288733,0.135142,2528
weighted avg,0.625745,0.212421,0.168120,2528


knn


,precision,recall,f1-score,support
Left,0.674419,0.743590,0.707317,351
OffAway,0.895046,0.897973,0.896507,1529
OnAway,0.489362,0.438095,0.462312,105
Right,0.742690,0.701657,0.721591,543
accuracy,0.815269,0.815269,0.815269,0
macro avg,0.700379,0.695329,0.696932,2528
weighted avg,0.814838,0.815269,0.814634,2528


In [154]:
#Cross Validation for Ada using first attempt as train, second attempt as test
twoset_test(a1, a2)

,precision,recall,f1-score,support
Left,0.577840,0.285417,0.382101,2887
OffAway,0.520263,0.745591,0.612872,4253
OnAway,0.050802,0.058282,0.054286,326
Right,0.474431,0.400581,0.434389,2756
accuracy,0.500685,0.500685,0.500685,0
macro avg,0.405834,0.372468,0.370912,10222
weighted avg,0.509195,0.500685,0.481759,10222


In [104]:
#Brian
data_dir="C:/Users/tsb7592/Box/Baby Toolbox Operations/Domain Teams/Language/Tobii Pilot/Tobii Verbose Files/Brian"
os.chdir(data_dir)
bdf=pd.read_json('A87CEC62-214F-4B81-A3CB-45B1DA445644@845BDD10-B9AF-4592-AA4E-0CCE76A0E8CD@00026@2021-05-12-00.13.17.121@LookListening@Brian@VerboseGazeData.json')

In [155]:
b1  = combine_tsv_json(tdf, bdf, 'BrianP2', k24)

In [168]:
#Cross Validation for Brian
own_test(b1)

RandomForest


,precision,recall,f1-score,support
Left,0.925613,0.901977,0.913642,1214
OffAway,0.571429,0.487805,0.526316,41
OnAway,0.863309,0.895522,0.879121,536
Right,0.898826,0.912844,0.905781,1090
accuracy,0.898993,0.898993,0.898993,0
macro avg,0.814794,0.799537,0.806215,2881
weighted avg,0.898846,0.898993,0.898733,2881


SupportVector


,precision,recall,f1-score,support
Left,0.909091,0.864909,0.886450,1214
OffAway,0.000000,0.000000,0.000000,41
OnAway,0.762951,0.906716,0.828645,536
Right,0.891644,0.890826,0.891235,1090
accuracy,0.870184,0.870184,0.870184,0
macro avg,0.640921,0.665613,0.651582,2881
weighted avg,0.862364,0.870184,0.864890,2881


NaiveBayes


,precision,recall,f1-score,support
Left,0.824924,0.888797,0.855670,1214
OffAway,0.000000,0.000000,0.000000,41
OnAway,0.784038,0.623134,0.694387,536
Right,0.856146,0.900917,0.877962,1090
accuracy,0.831309,0.831309,0.831309,0
macro avg,0.616277,0.603212,0.607005,2881
weighted avg,0.817390,0.831309,0.821920,2881


knn


,precision,recall,f1-score,support
Left,0.918760,0.903624,0.911130,1214
OffAway,0.571429,0.390244,0.463768,41
OnAway,0.845343,0.897388,0.870588,536
Right,0.901835,0.901835,0.901835,1090
accuracy,0.894481,0.894481,0.894481,0
macro avg,0.809342,0.773273,0.786830,2881
weighted avg,0.893755,0.894481,0.893704,2881
